# TASK 2: Calculating the value of a contract

## Setting up price prediction model for Task 2

In [12]:
# Importing Libaries
import pandas as pd
from prophet import Prophet
import datetime as dt

# Reading in datasets
nat_gas = pd.read_csv("Nat_Gas.csv")
# Convert the 'Dates' column to datetime format
nat_gas['Dates'] = pd.to_datetime(nat_gas['Dates'], format='%m/%d/%y')
# Ordering Data
nat_gas.set_index('Dates', inplace=True)

# Prepare data for Prophet
prophet_data = nat_gas.reset_index()[['Dates', 'Prices']].rename(columns={'Dates': 'ds', 'Prices': 'y'})

# Initialise and fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(prophet_data)

nat_gas.reset_index(inplace = True)

def get_price_predictions(dates):
    # Check if dates is a list
    if not isinstance(dates, list):
        raise ValueError("The input should be a list of dates.")
    
    # Initialize an empty list to store predictions
    predictions = []
    
    # Convert list of dates to a DataFrame for Prophet
    df_dates = pd.DataFrame({'ds': [dt.datetime.strptime(date, "%Y-%m-%d") if isinstance(date, str) else date for date in dates]})
    
    # Ensure 'ds' column is in the correct datetime format
    df_dates['ds'] = pd.to_datetime(df_dates['ds'])
    
    # Predict using Prophet model
    forecast = prophet_model.predict(df_dates)
    
    # Extract and rename columns of interest
    forecast_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    
    # Rename columns
    forecast_df = forecast_df.rename(
        columns={
            "ds": "Dates",
            "yhat": "Predicted Price",
            "yhat_lower": "Lower Bound Uncertainty Interval",
            "yhat_upper": "Upper Bound Uncertainty Interval"
        }
    )
    
    return forecast_df

16:33:52 - cmdstanpy - INFO - Chain [1] start processing
16:33:52 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
dates = ["2024-09-01", "2024-10-01", dt.datetime(2024, 11, 1), "2026-01-14"]
forecast_df = get_price_predictions(dates)
forecast_df


,Dates,Predicted Price,Lower Bound Uncertainty Interval,Upper Bound Uncertainty Interval
0,2024-09-01,11.926671,11.755825,12.102350
1,2024-10-01,11.836000,11.644049,12.004872
2,2024-11-01,11.941295,11.754883,12.130328
3,2026-01-14,13.559415,13.372011,13.741288


for a single injection and withdraw date the price of the contract can be detemrined as follows:

- sell price - buy price
- minus storage fee (monthly storage fee * num_of_months stored)
- minus injection_withdraw_cost (rate/1000000 * volume)
- minus transport cost * 2 - fixed
- contract_value = (sell price - buy price) - storage_fee - injection)withdraw_cost - transport_cost

need to ensure the injection dates and withdrawal dates are a list of dates

# Task 2

In [57]:
import pandas as pd

def contract_value(injection_dates, withdrawal_dates, injection_rate, 
                   max_storage_volume, monthly_storage_costs, injection_withdrawal_costs):

    value_of_contract = 0
    avg_num_day_per_month = 30.42
    
    for i in range(len(injection_dates)):

        # Get prices for injection and withdrawal dates
        injection_df = get_price_predictions([injection_dates[i]])
        injection_price = injection_df['Predicted Price'].values[0]
        print(f"injection_price: {injection_price:,.2f}")
        
        withdrawal_df = get_price_predictions([withdrawal_dates[i]])
        withdrawal_price = withdrawal_df['Predicted Price'].values[0]
        print(f"withdrawal_price: {withdrawal_price:,.2f}")

        # Calculate the number of months in storage
        months_in_store = round((pd.to_datetime(withdrawal_dates[i]) - pd.to_datetime(injection_dates[i])).days / avg_num_day_per_month)
        print(f"months_in_store: {months_in_store}")
        
        # Calculate the total volume in storage
        total_injected_volume = min(months_in_store * injection_rate, max_storage_volume)
        print(f"total_injected_volume: {total_injected_volume:,}")

        # Calculate costs and revenues
        cost_of_injection = (total_injected_volume * injection_price) + (total_injected_volume / 1000000 * injection_withdrawal_costs) + transport_cost
        print(f"cost_of_injection: {cost_of_injection:,.2f}")
        
        revenue_from_sale = (total_injected_volume * withdrawal_price) - (total_injected_volume / 1000000 * injection_withdrawal_costs)
        print(f"revenue_from_sale: {revenue_from_sale:,.2f}")
        
        total_storage_cost = months_in_store * monthly_storage_costs
        print(f"total_storage_cost: {total_storage_cost:,.2f}")

        # Update the total value of the contract
        value_of_contract += (revenue_from_sale - cost_of_injection - total_storage_cost)

    # Format and return the final contract value
    return f"Contract Value: {value_of_contract:,.2f}"


In [59]:
# Parameters
monthly_storage_costs = 100000
transport_cost = 50000
injection_rate = 1000000
injection_withdrawal_costs = 10000
max_storage_volume = 5000000
injection_dates = ["2024-10-31"]
withdrawal_dates = ["2025-02-28"]

In [61]:
contract_value(injection_dates, withdrawal_dates, injection_rate, 
                   max_storage_volume, monthly_storage_costs, injection_withdrawal_costs)

injection_price: 11.96
withdrawal_price: 12.88
months_in_store: 4
total_injected_volume: 4,000,000
cost_of_injection: 47,931,013.57
revenue_from_sale: 51,461,145.96
total_storage_cost: 400,000.00


'Contract Value: 3,130,132.39'